In [50]:
import os
import numpy as np
import torch as tr

from CSWSEM import *
from matplotlib import pyplot as plt

In [4]:
gs_name = 'absem'

In [25]:
from glob import glob as glob
import pandas as pd

def make_gsdf(gs_name):
  gs_dir = "gsdata/%s/"%gs_name
  fpathL = glob(gs_dir+'*')
  seed_df_L = []
  for fpath in fpathL:
    condition = fpath.split('/')[-1].split('__')[1].split('_')[1]
    seed_df = pd.read_csv(fpath)
    seed_df.loc[:,'condition'] = condition
    seed_df_L.append(seed_df)
  gsdf = pd.concat(seed_df_L)
  gsdf.index = np.arange(len(gsdf))
  return gsdf

In [26]:
gsdf = make_gsdf(gs_name)

In [69]:
paramL = ['condition','nosplit','learn_rate','lmda','alfa']
loss_df = gsdf.groupby(paramL+['trial']).mean().loss
df_group = loss_df.groupby(paramL)

for param_tuple,mean_loss in df_group:
  param_str = '-'.join([str(i) for i in param_tuple])
  plt.figure(figsize=(8,3))
  plt.plot(1-mean_loss.values)
  plt.title(param_str)
  plt.ylim(0,1)
  plt.savefig('figures/gs-%s/loss-%s.png'%(gs_name,param_str))
  plt.close('all')